In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext


sc = SparkContext('local', 'Spark SQL') 
spark = SQLContext(sc)

In [2]:
from pyspark.sql.functions import explode, col

In [5]:
source_df = spark.read.option("multiline", "true").json("persons.json")

In [6]:
source_df.printSchema()

root
 |-- persons: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- age: long (nullable = true)
 |    |    |-- cars: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- models: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |-- name: string (nullable = true)



In [11]:
# Explode all persons into different rows
persons = source_df.select(explode("persons").alias("persons"))
display(persons)
persons.printSchema()
# Explode all car brands into different rows
persons_cars = persons.select(
   col("persons.name").alias("persons_name")
 , col("persons.age").alias("persons_age")
 , explode("persons.cars").alias("persons_cars_brands")
 , col("persons_cars_brands.name").alias("persons_cars_brand")
)
display(persons_cars)
persons_cars.printSchema()
# Explode all car models into different rows
persons_cars_models = persons_cars.select(
   col("persons_name")
 , col("persons_age")
 , col("persons_cars_brand")
 , explode("persons_cars_brands.models").alias("persons_cars_model")
)
display(persons_cars_models)
persons_cars_models.printSchema()

DataFrame[persons: struct<age:bigint,cars:array<struct<models:array<string>,name:string>>,name:string>]

root
 |-- persons: struct (nullable = true)
 |    |-- age: long (nullable = true)
 |    |-- cars: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- models: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- name: string (nullable = true)
 |    |-- name: string (nullable = true)



DataFrame[persons_name: string, persons_age: bigint, persons_cars_brands: struct<models:array<string>,name:string>, persons_cars_brand: string]

root
 |-- persons_name: string (nullable = true)
 |-- persons_age: long (nullable = true)
 |-- persons_cars_brands: struct (nullable = true)
 |    |-- models: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- name: string (nullable = true)
 |-- persons_cars_brand: string (nullable = true)



DataFrame[persons_name: string, persons_age: bigint, persons_cars_brand: string, persons_cars_model: string]

root
 |-- persons_name: string (nullable = true)
 |-- persons_age: long (nullable = true)
 |-- persons_cars_brand: string (nullable = true)
 |-- persons_cars_model: string (nullable = true)



In [9]:
display(persons_cars_models)

DataFrame[persons_name: string, persons_age: bigint, persons_cars_brand: string, persons_cars_model: string]